# Yunjue Agent Benchmark - A100 GPU Acceleration

This notebook runs the financial agent benchmark using local LLM inference on A100 80G.

**Requirements**: Colab Pro+ with A100 GPU runtime

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/YOUR_REPO/insitu-finance-agent.git
%cd insitu-finance-agent/fin_evo_agent

In [ ]:
# Install base dependencies
!pip install -q -r requirements.txt

In [ ]:
# Install GPU dependencies for local LLM
!pip install -q -r requirements-gpu.txt

## 2. Option A: vLLM Server (Recommended)

vLLM provides OpenAI-compatible API with optimal GPU utilization.

In [ ]:
# Install vLLM
!pip install -q vllm>=0.4.0

In [ ]:
import subprocess
import time
import os

# Choose model - Qwen2.5-Coder is excellent for code generation
# For A100 80G, we can run 32B+ models
MODEL_ID = "Qwen/Qwen2.5-Coder-32B-Instruct"  # Best for code
# Alternative: "deepseek-ai/deepseek-coder-33b-instruct"
# Alternative: "Qwen/Qwen2.5-72B-Instruct"  # For more reasoning

# Start vLLM server in background
vllm_process = subprocess.Popen(
    [
        "python", "-m", "vllm.entrypoints.openai.api_server",
        "--model", MODEL_ID,
        "--host", "0.0.0.0",
        "--port", "8000",
        "--tensor-parallel-size", "1",  # Single A100
        "--gpu-memory-utilization", "0.90",
        "--max-model-len", "8192",
        "--trust-remote-code",
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print("Starting vLLM server...")
time.sleep(60)  # Wait for model to load
print("vLLM server should be ready!")

In [ ]:
# Test vLLM endpoint
!curl -s http://localhost:8000/v1/models | python -m json.tool

In [ ]:
# Configure environment for local LLM
import os

os.environ["LLM_TYPE"] = "local"
os.environ["LLM_BASE_URL"] = "http://localhost:8000/v1"
os.environ["LLM_MODEL"] = MODEL_ID
os.environ["API_KEY"] = "not-needed"  # vLLM doesn't require API key

## 2. Option B: Direct Transformers (Alternative)

If vLLM doesn't work, use direct transformers integration.

In [ ]:
# Skip this cell if using vLLM
# This is for direct transformers usage

import os
os.environ["LLM_TYPE"] = "transformers"
os.environ["LLM_MODEL_ID"] = "Qwen/Qwen2.5-Coder-32B-Instruct"
os.environ["LLM_QUANTIZATION"] = "none"  # A100 80G can handle full precision
# Use "4bit" or "8bit" if you want faster inference with slight quality tradeoff

## 3. Initialize Database

In [ ]:
!python main.py --init

## 4. Run Benchmark

In [ ]:
import time

start = time.time()
!python benchmarks/run_eval.py --config cold_start --run-id colab_a100_run
elapsed = time.time() - start

print(f"\n{'='*60}")
print(f"Total benchmark time: {elapsed:.1f}s ({elapsed/60:.1f} minutes)")
print(f"{'='*60}")

In [ ]:
# View results
import json
from pathlib import Path

results_file = Path("benchmarks/results/colab_a100_run.json")
if results_file.exists():
    with open(results_file) as f:
        results = json.load(f)
    
    print(f"Pass Rate: {results['summary']['pass_rate']*100:.1f}%")
    print(f"Passed: {results['summary']['passed']}/{results['summary']['total']}")
    print(f"\nBy Category:")
    for cat, stats in results['summary'].get('by_category', {}).items():
        print(f"  {cat}: {stats['passed']}/{stats['total']}")
else:
    print("Results file not found")

## 5. GPU Memory Monitor

In [ ]:
# Check GPU memory during/after benchmark
!nvidia-smi --query-gpu=name,memory.used,memory.total,utilization.gpu --format=csv

## 6. Cleanup

In [ ]:
# Stop vLLM server if running
try:
    vllm_process.terminate()
    print("vLLM server stopped")
except:
    pass

## Performance Comparison

| Setup | Model | Time (20 tasks) | Pass Rate |
|-------|-------|-----------------|----------|
| API (Qwen3-Max) | qwen3-max | ~180s | 85% |
| A100 + vLLM | Qwen2.5-Coder-32B | ~60s | TBD |
| A100 + Transformers | Qwen2.5-Coder-32B | ~90s | TBD |